<h2> 異分野共著　process I : Make ID list </h2>

In [92]:
#! Python3
# -*- coding: utf-8 -*-
# 異分野共著　process I : Make ID list

import csv, os, glob
import pandas  as pd
import numpy as np


<h3>0.事前準備</h3>

In [93]:
#複数ファイルがある場合: fromSciValというフォルダを作り、SciValからDLしたファイルをそのまま入れておく
#DLした項目によって、下記を変更
col_names =  ['Title', 'Authors', 'Number of Authors', 'Scopus Author Ids', 'Year', 'Scopus Source title', 'SNIP 2017', 'Citations',
       'Field-Weighted Outputs in Top Citation Percentiles, per percentile', 'DOI', 'Publication-type', 'EID',
       'All Science Journal Classification (ASJC)']

#fromSciVal内にあるファイルの名を変更、一つのリストにまとめる
publicationsAll = pd.DataFrame(index=[], columns=col_names)
# カレントディレクトリの全ファイルをループする
for csv_file in os.listdir('fromSciVal/.'):
    if not csv_file.endswith('.csv'):
        continue # CSVファイルでなければスキップ

    print('Change name: ' + csv_file + '...')
    df = pd.read_csv('fromSciVal/' + csv_file, header = None, dtype ='object',names = col_names)
    dfname = str(df.iat[0,1] + "_" +df.iat[1,1])
    os.rename('fromSciVal/' + csv_file, 'fromSciVal/' + dfname + '_190207.csv')
    df2 = df.dropna(axis = 0, subset=['All Science Journal Classification (ASJC)'])
    df3= df2[df2['Title'] != 'Title']
#    df3['Scopus Author Ids'] =  df3['Scopus Author Ids'].str.replace("|",",")
    print(df3['Scopus Author Ids'])
    
    
    publicationsAll = pd.concat([publicationsAll, df3], ignore_index=True)

# 出力するファイル名を指定する。
publicationsAll.to_csv('Publications_KU2020.csv', index = False)

Change name: Publications at Kyoto University_2020_190207.csv...
13      55553077000| 57213186397| 57160212700| 9845935500
14      57191752336| 57204598303| 35254533400| 2592812...
15      55917949000| 56140399700| 35254533400| 7004006...
16      57218886871| 7103161459| 57205227392| 15045889...
17                               35397669200| 55728708900
                              ...                        
8802    57193431831| 56492963100| 55568512146| 6507202...
8803                 57223143923| 36634427800| 7202931420
8804    6701596602| 56011428100| 7004219270| 660339844...
8805    57210727646| 37067554200| 38061869500| 5576017...
8806                 57216585956| 57203369625| 7006122085
Name: Scopus Author Ids, Length: 8794, dtype: object



<h3> 1. publicationsAllより、重複削除、単著論文を除外、HAPは著者情報が500までしかDLされないが、とりあえずそのまま（APIで確認し直す）</h3>

In [94]:
publicationsAll['Number of Authors'] = publicationsAll['Number of Authors'].astype(int)
uniquepubdf = publicationsAll.drop_duplicates()
targetdf = uniquepubdf[uniquepubdf['Number of Authors'] > 1]
print('all publications:' + str(len(uniquepubdf)))
print('Target publications (単著以外):' + str(len(targetdf)))

all publications:8794
Target publications (単著以外):8384


In [95]:
targetdf.columns

Index(['Title', 'Authors', 'Number of Authors', 'Scopus Author Ids', 'Year',
       'Scopus Source title', 'SNIP 2017', 'Citations',
       'Field-Weighted Outputs in Top Citation Percentiles, per percentile',
       'DOI', 'Publication-type', 'EID',
       'All Science Journal Classification (ASJC)'],
      dtype='object')


<h3>2. Author ID を抽出(重複あり)、数を確認</h3>

In [96]:
aIds = []
nofaIds = 0
for i, row in targetdf.iterrows():
    author = [row['Scopus Author Ids']]
    author = author[0].split('| ')
    aIds.extend(author)
    if row['Number of Authors'] > 500:
        nofa = 500
    else:
        nofa = row['Number of Authors']
    nofaIds = nofaIds + nofa

    #HAP以外でIDの数とAuthor数が一致しない場合、そのscopusIDを出力（一応確認のため）
    if len(author) != nofa:
        print(row['EID'])
    else:
        continue
        
aIdsdf = pd.DataFrame(aIds, columns=['Author ID'])

#Authorの数とIDの数が一致するか確認
print('Author Ids count: ' + str(len(aIdsdf)))
print('Number of Authors: ' + str(nofaIds))

20315
21206
29949
21100201068
16687
29949
17382
21100370017
15316
16767
29083
21100779241
29150
28371
15536
29150
20313
19897
19064
23062
25453
22305
18429
21100205926
23774
23062
21100924827
25453
28218
24548
26662
14639
23825
29150
22415
21100829284
25295
26000
12531
21100200805
21100779241
21100970268
21100829284
12539
40067
23875
21100779241
21100779241
21100924827
23062
21101041514
21100211738
21100211738
23741
23181
29067
24863
21100779241
25453
21100779241
3900148506
29767
29150
17525
19194
23062
26662
23899
28541
21100200805
23062
40067
20344
29067
25453
26781
29794
23062
21100779241
40067
26781
21100220161
15894
25453
19417
16767
7600153101
21100779241
13213
20648
22416
25979
15857
25997
12391
40173
27608
28520
21100779241
20577
17382
22415
21100200805
23752
29482
26781
29970
26781
21524
144989
19800188003
22415
21100779241
21100786208
17600154902
20185
21100200805
28747
130053
21101018916
19196
25453
24863
15116
21100900345
20648
29954
26781
19194
13867
19700177504
29150
2110


<h3>3. Unique Author ID listを作る</h3>

In [97]:
uniqueaIds = aIdsdf.drop_duplicates()
uniqueaIds =uniqueaIds.astype(int)
print('Unique Author IDs: ' + str(len(uniqueaIds)))




Unique Author IDs: 46406


In [98]:
uniqueaIds.to_csv('UniqueAuthorIdsList.csv', index = False)